In [57]:
# find out all bars and pubs within 500 meters of a metro station in Australia
import pandas as pd
import geopandas as gpd

csv_file = 'Bars_and_pubs__with_patron_capacity.csv'
barspubs_df = pd.read_csv(csvPath) 

barspubs_gdf = gpd.GeoDataFrame(
    barspubs_df, geometry=gpd.points_from_xy(barspubs_df['x coordinate'],barspubs_df['y coordinate']))
barspubs_gdf.crs = 'EPSG:4326'
barspubs_gdf_reprojected = barspubs_gdf.to_crs('EPSG:7855')

metrostations_gdf = gpd.read_file('zip://metro_stations_accessbility.zip', layer = 'metro_stations_accessbility')
metrostations_gdf_reprojected = metrostations_gdf.to_crs('EPSG:7855')


In [58]:
# Set the buffer radius of 500m. EPSG:7855 is in meters
radius = 500

# We match QGIS's default buffer parameters
# https://shapely.readthedocs.io/en/latest/manual.html#constructive-methods
metrostations_gdf_reprojected['geometry'] = metrostations_gdf_reprojected.buffer(
    radius, resolution=5, cap_style=1, join_style=1, mitre_limit=2)

# As we want to do a spatial join, we need to dissolve the resulting buffers
# Otherwise we will get 1 feature for every intersecting buffer

# Create a new field with a value of 1, so we can simular a 'Dissolve All'
metrostations_gdf_reprojected['dissolvefield'] = 1

dissolved_buffers = metrostations_gdf_reprojected.dissolve(by='dissolvefield')

In [59]:
# Now we do a spatial join. Select 'inner' join so we get all pubs intersecting buffers
watering_holes = gpd.sjoin(barspubs_gdf_reprojected, dissolved_buffers, how='inner', op='intersects')
watering_holes.shape

(2147, 17)

In [61]:
output_file = 'watering_holes.gpkg'
watering_holes.to_file(driver='GPKG', filename=output_file, encoding='utf-8')